In [1]:
import pyodbc
import pandas as pd
import boto3
from boto3.session import Session
import warnings
warnings.filterwarnings("ignore")

In [2]:
file_credenciais = r'C:\Users\GN6006\Documents\Data_PO\CDH\credenciais_cdh.xlsx.'
df_credenciais = pd.read_excel(file_credenciais)

In [3]:
# informações para login no CDH
username = df_credenciais.iloc[0,1]
AccessKeyId = df_credenciais.iloc[1,1]
SecretKey = df_credenciais.iloc[2,1]
region = df_credenciais.iloc[3,1]
bucketname = df_credenciais.iloc[4,1]
token = df_credenciais.iloc[5,1]

In [4]:
# conecta ao excel que contém login e a senha do BD
caminho = r'C:\Users\GN6006\Documents\Data_PO\Alimentadora'
df_login = pd.read_excel(caminho+'\LoginBD.xlsx')

In [5]:
# informações para conexão ao banco
server = '10.179.6.16\RELATORIO,58298'
database = 'relatorio'
username = df_login.iloc[0,1]
password = df_login.iloc[1,1]

In [6]:
# conexão ao banco
cnxn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
)

In [7]:
# consulta sql
cursor = cnxn.cursor()

vwTabela = pd.read_sql_query(
    'select\
        pldh.DataHora, pldsubmerc.* \
    from \
        vwPLDHorario as pldh \
    inner join \
    (select \
        pldsm.Id, pldsm.IdPLDHorario, pldsm.IdSubMercados, pldsm.Valor, sm.Descricao as submercado \
    from \
        vwPLDHorarioSubMercado as pldsm inner join vwSubMercados as sm \
    on \
        pldsm.IdSubMercados = sm.Id) as pldsubmerc \
    on \
        pldh.Id = pldsubmerc.IdPLDHorario \
    order by \
        pldh.DataHora desc'
    , cnxn)

In [8]:
data = []
hora = []
ano = []
mes = []
dia = []

for element in list(vwTabela['DataHora'].apply(lambda x: str(x).split(' '))):
    data.append(element[0])
    hora.append(element[1])
    
vwTabela['Data'] = data

for element in list(vwTabela['Data'].apply(lambda x: str(x).split("-"))):
    ano.append(element[0])
    mes.append(element[1])
    dia.append(element[2])
    
vwTabela['Hora'] = hora


vwTabela = vwTabela.iloc[:,1:]
vwTabela

,Id,IdPLDHorario,IdSubMercados,Valor,submercado,Data,Hora
0,188637,47160,2,55.70,Sudeste - Centro Oeste,2022-02-17,23:00:00
1,188638,47160,3,55.70,Sul,2022-02-17,23:00:00
2,188639,47160,1,55.70,Nordeste,2022-02-17,23:00:00
3,188640,47160,4,55.70,Norte,2022-02-17,23:00:00
4,188633,47159,2,55.70,Sudeste - Centro Oeste,2022-02-17,22:00:00
...,...,...,...,...,...,...,...
74779,110696,27674,4,285.97,Norte,2020-01-01,01:00:00
74780,110689,27673,2,286.03,Sudeste - Centro Oeste,2020-01-01,00:00:00
74781,110690,27673,3,286.03,Sul,2020-01-01,00:00:00
74782,110691,27673,1,305.92,Nordeste,2020-01-01,00:00:00


In [9]:
# salvando excel
vwTabela.to_csv(r'C:\Users\GN6006\Documents\Data_PO\CatalogoPLDh\vwTabela.csv', sep=";")
arquivo_local = r'C:\Users\GN6006\Documents\Data_PO\CatalogoPLDh\vwTabela.csv'
print('Salvou o CSV')

Salvou o CSV


In [10]:
# iniciando o boto3
s3_client = boto3.client('s3',
    aws_access_key_id=AccessKeyId,
    aws_secret_access_key=SecretKey,
    aws_session_token=token,
    region_name=region
)

In [11]:
# fazendo upload do arquivo
s3_client.upload_file(arquivo_local, bucketname, 'vwTabela.csv')